# Agents for Reliable Multi‑Agent AI

In this notebook, we will explore into the concept of MCP, its advantages in multi-agent orchestration, and the process of integrating MCP servers into the Agno agent framework.

In [1]:
import configparser
def get_aws_credentials_from_file():
    try:
        # Path to the AWS credentials file
        aws_credentials_path = os.path.expanduser("~/.aws/credentials")
        
        # Create a ConfigParser object
        config = configparser.ConfigParser()
        
        # Read the credentials file
        config.read(aws_credentials_path)
        
        # Extract the access key ID and secret access key from the [default] profile
        if 'default' in config:
            aws_access_key_id = config['default'].get('aws_access_key_id')
            aws_secret_access_key = config['default'].get('aws_secret_access_key')
            return aws_access_key_id, aws_secret_access_key
        else:
            print("No [default] profile found in credentials file")
            return None, None
            
    except Exception as e:
        print(f"Error reading AWS credentials: {e}")
        return None, None

### Self served vllm models

In [2]:
from agno.models.vllm import vLLM
model_qwen = vLLM(base_url="http://agent.cavatar.info:8081/v1", api_key="EMPTY", id="Qwen/Qwen3-30B-A3B",temperature=0.2, top_p=0.90, presence_penalty=1.45)
model_gemma = vLLM(base_url="http://infs.cavatar.info:8081/v1", api_key="EMPTY", id="alfredcs/gemma-3-27b-grpo-med-merged",temperature=0.2, top_p=0.90, presence_penalty=1.45)

### Bedrock integreation

In [3]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat  # could be OpenRouter as well
from openai import OpenAI
import os
from agno.tools.mcp import MCPTools
from agno.agent import Agent
from agno.models.openai import OpenAIChat

import asyncio
from textwrap import dedent
from agno.knowledge.url import UrlKnowledge
#from agno.models.openai import OpenAIChat
from agno.models.aws.bedrock import AwsBedrock
from agno.models.aws.claude import Claude
from agno.playground import Playground, serve_playground_app
from agno.storage.sqlite import SqliteStorage
from agno.team import Team
from agno.tools.tavily import TavilyTools
from agno.tools.knowledge import KnowledgeTools
from agno.tools.reasoning import ReasoningTools
from agno.tools.thinking import ThinkingTools
from agno.tools.yfinance import YFinanceTools
#from agno.vectordb.lancedb import LanceDb, SearchType
from agno.vectordb.chroma import ChromaDb

model_id_c37 = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model_id_h35 = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
model_id_c35 = "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
model_id_nova = 'us.amazon.nova-lite-v1:0'
model_id_ds = 'us.deepseek.r1-v1:0'

#os.environ["OPENAI_API_KEY"] = openai_api_key = os.getenv("bedrock_api_token")
#os.environ["OPENAI_BASE_URL"] = openai_base_url = os.getenv("bedrock_api_url")
os.environ["TAVILY_API_KEY"] = tavily_api_key = os.getenv("tavily_api_token")
os.environ["AWS_ACCESS_KEY_ID"], os.environ["AWS_SECRET_ACCESS_KEY"] = get_aws_credentials_from_file()

# Standard LLM model (e.g., GPT-4 via OpenRouter/OpenAI)
#model = OpenAIChat(id=model_id_nova)  # assumes API key set via env
#model = Claude(id=model_id_c37, aws_region="us-west-2")  # assumes API key set via env
model_nova = AwsBedrock(id=model_id_nova)
model_c37 = AwsBedrock(id=model_id_c37)
model_h35 = AwsBedrock(id=model_id_h35)
model_ds = AwsBedrock(id=model_id_ds)

In [4]:
## OpenAI/Bedrock API
from langchain_openai import ChatOpenAI
from agno.models.openai import OpenAIChat
os.environ["OPENAI_API_KEY"] = api_key = os.getenv("bedrock_api_token")
os.environ["OPENAI_BASE_URL"] = base_url = os.getenv("bedrock_api_url")

model_openai_br = ChatOpenAI(
    model=model_id_c37,
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=5,
    #api_key=os.getenv("bedrock_api_token"),  # if you prefer to pass api key in directly instaed of using env vars
    #base_url=os.getenv("bedrock_api_url"),
    # organization="...",
    # other params...
)
model_openai = OpenAIChat(id="gpt-4o-mini", api_key=os.getenv("openai_api_token"), base_url="https://api.openai.com/v1/")

### A quick test -- **Optional**

In [53]:
## [Optional] Fo rtest only
agent = Agent(
    name="CodeAssistant",
    role="An AI agent that answers programming questions.",
    model=model_openai,
    tools=[],  # no external tools, just the LLM
    instructions="You are a seasoned assistant. Answer questions professionally and precisely.",
    stream_intermediate_steps=True,
)
response = agent.run("Who is Elon Musk?")
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.agno.com/v1/telemetry/agent/run/create "HTTP/1.1 200 OK"


RunResponse(content='Elon Musk is a prominent entrepreneur and business magnate known for his involvement in multiple groundbreaking companies and ventures. He was born on June 28, 1971, in Pretoria, South Africa. Musk is the CEO and lead designer of SpaceX, a company focused on space exploration and satellite technology; he is also the CEO and product architect of Tesla, Inc., which specializes in electric vehicles and renewable energy solutions. Additionally, he co-founded Neuralink, a neurotechnology company, and The Boring Company, which focuses on tunnel construction and infrastructure.\n\nMusk gained attention for his vision of sustainable energy and colonizing Mars, and he has been a key figure in popularizing electric vehicles. His ventures often revolve around innovative technologies and ambitious future goals, making him a significant figure in the technology and automotive industries.', content_type='str', thinking=None, reasoning_content=None, messages=[Message(role='system

### Local store and DB definitionation

In [6]:
agent_storage_file: str = "tmp/agents.db"
image_agent_storage_file: str = "tmp/image_agent.db"

db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

In [7]:
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    agent_id="finance-agent",
    model=model_gemma,
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            company_info=True,
            company_news=True,
        )
    ],
    instructions=["Always use tables to display data"],
    storage=SqliteStorage(
        table_name="finance_agent", db_file=agent_storage_file, auto_upgrade_schema=True
    ),
    add_history_to_messages=True,
    num_history_responses=5,
    add_datetime_to_instructions=True,
    markdown=True,
)

#### Cot Agent

In [8]:
cot_agent = Agent(
    name="Chain-of-Thought Agent",
    role="Answer basic questions",
    agent_id="cot-agent",
    model=model_ds,
    storage=SqliteStorage(
        table_name="cot_agent", db_file=agent_storage_file, auto_upgrade_schema=True
    ),
    add_history_to_messages=True,
    num_history_responses=3,
    add_datetime_to_instructions=True,
    markdown=True,
    reasoning=True,
)

#### ToT Agent

In [115]:
from agno.core.plan import PlanStep, Planner
from tree_of_thoughts import TreeOfThoughts, OptimizedAgent

class AgnoTreeOfThoughtsAgent(Agent):
    def __init__(self, model=model_ds, **kwargs):
        super().__init__(**kwargs)
        self.tot = TreeOfThoughts(
            agent=OptimizedAgent(model),
            strategy="BFS",
            evaluation_strategy="value",
            k=3,
            T=3,
            b=3
        )

    def plan(self, problem: str):
        return [PlanStep(description="Solve with ToT", metadata={"problem": problem})]

    def act(self, step: PlanStep):
        result = self.tot.solve(step.metadata["problem"])
        return result


tot_agent = AgnoTreeOfThoughtsAgent()

ModuleNotFoundError: No module named 'agno.core'

#### REasoning agent

In [9]:
reasoning_model_agent = Agent(
    name="Reasoning Model Agent",
    role="Reasoning about Math",
    agent_id="reasoning-model-agent",
    model=model_nova,
    reasoning_model=model_ds,
    instructions=["You are a reasoning agent that can reason about math."],
    show_tool_calls=True,
    markdown=True,
    debug_mode=True,
    storage=SqliteStorage(
        table_name="reasoning_model_agent",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
)

In [10]:
reasoning_tool_agent = Agent(
    name="Reasoning Tool Agent",
    role="Answer basic questions",
    agent_id="reasoning-tool-agent",
    model=model_nova,
    storage=SqliteStorage(
        table_name="reasoning_tool_agent",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
    add_history_to_messages=True,
    num_history_responses=3,
    add_datetime_to_instructions=True,
    markdown=True,
    tools=[ReasoningTools()],
)


In [11]:
web_agent = Agent(
    name="Web Search Agent",
    role="Handle web search requests",
    model=model_gemma,
    agent_id="web_agent",
    tools=[TavilyTools()],
    instructions="Always include sources",
    add_datetime_to_instructions=True,
    storage=SqliteStorage(
        table_name="web_agent",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
    stream=True,
    stream_intermediate_steps=True,
)

### MCP server fintergration

In [12]:
from agno.tools.mcp import MCPTools
from textwrap import dedent
sequential_thinking_mcp_tools = MCPTools(
            command="npx -y @modelcontextprotocol/server-sequential-thinking"
        ) 
stream_mcp_arxiv = MCPTools(
            url="http://infs.cavatar.info:8084/mcp", transport="streamable-http", timeout_seconds=20
        ) 

In [13]:
finance_agent_2 = Agent(
    name="Finance Agent2",
    role="Get financial data with tool specified by a MCP server",
    agent_id="finance-agent-2",
    model=model_qwen,
    tools=[
        sequential_thinking_mcp_tools,
        stream_mcp_arxiv,
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            company_info=True,
            company_news=True,
        )
    ],
    instructions=dedent("""\
                ## Using the think tool
                Before taking any action or responding to the user after receiving tool results, use the think tool as a scratchpad to:
                - List the specific rules that apply to the current request
                - Check if all required information is collected
                - Verify that the planned action complies with all policies
                - Iterate over tool results for correctness

                ## Rules
                - Its expected that you will use the think tool generously to jot down thoughts and ideas.
                - Use tables where possible\
                """),
    storage=SqliteStorage(
        table_name="finance_agent", db_file=agent_storage_file, auto_upgrade_schema=True
    ),
    add_history_to_messages=True,
    num_history_responses=5,
    add_datetime_to_instructions=True,
    markdown=True,
)

### Thinking agno tool

In [130]:
finance_thinking_prompt = """\
        You are a seasoned Wall Street analyst with deep expertise in market analysis! 📊

        Follow these steps for comprehensive financial analysis:
        1. Market Overview
           - Latest stock price
           - 52-week high and low
        2. Financial Deep Dive
           - Key metrics (P/E, Market Cap, EPS)
        3. Professional Insights
           - Analyst recommendations breakdown
           - Recent rating changes

        4. Market Context
           - Industry trends and positioning
           - Competitive analysis
           - Market sentiment indicators

        Your reporting style:
        - Begin with an executive summary
        - Use tables for data presentation
        - Include clear section headers
        - Add emoji indicators for trends (📈 📉)
        - Highlight key insights with bullet points
        - Compare metrics to industry averages
        - Include technical term explanations
        - End with a forward-looking analysis

        Risk Disclosure:
        - Always highlight potential risk factors
        - Note market uncertainties
        - Mention relevant regulatory concerns\
    """

ai_thinking_prompt = """
        You are a seasoned AI expert specializing in analyzing research papers from arXiv and code repositories from GitHub! 📊

        Follow these steps for comprehensive academic and technical analysis:
        1. Research Evaluation
           - Paper methodology assessment
           - Technical innovation analysis
           - Implementation feasibility
           - Code quality assessment
        
        2. Deep Technical Dive
           - Key algorithms and architectures
           - Mathematical formulations
           - Technical limitations
           - Experimental design critique
        
        3. Professional Insights
           - Research impact and significance
           - Implementation challenges
           - Comparative analysis with SOTA
           - Potential applications
        
        4. Technical Context
           - Related research connections
           - Code architecture evaluation
           - Algorithm complexity analysis
           - Theoretical foundations
        
        Your analytical style:
        - Begin with an executive summary of key findings
        - Use code blocks for illustrating implementations
        - Include clear technical section headers
        - Add evaluation indicators for strengths/weaknesses (🔍 ⚠️)
        - Highlight technical insights with bullet points
        - Compare with established technical benchmarks
        - Include technical term explanations
        - End with future research directions
        
        Technical Disclosure:
        - Always highlight computational complexity concerns
        - Note implementation challenges
        - Mention relevant theoretical limitations
        - Address reproducibility considerations
    """

thinking_prompt = ai_thinking_prompt

thinking_tool_agent = Agent(
    name="Thinking Tool Agent",
    agent_id="thinking_tool_agent",
    model=model_gemma,
    tools=[ThinkingTools(add_instructions=True), YFinanceTools(enable_all=True)],
    instructions=dedent(thinking_prompt),
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
    stream_intermediate_steps=True,
    storage=SqliteStorage(
        table_name="thinking_tool_agent",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
)

DEBUG Function: think registered with thinking_tools

DEBUG Function: get_current_stock_price registered with yfinance_tools

DEBUG Function: get_company_info registered with yfinance_tools

DEBUG Function: get_stock_fundamentals registered with yfinance_tools

DEBUG Function: get_income_statements registered with yfinance_tools

DEBUG Function: get_key_financial_ratios registered with yfinance_tools

DEBUG Function: get_analyst_recommendations registered with yfinance_tools

DEBUG Function: get_company_news registered with yfinance_tools

DEBUG Function: get_technical_indicators registered with yfinance_tools

DEBUG Function: get_historical_stock_prices registered with yfinance_tools

In [15]:
from agno.embedder.openai import OpenAIEmbedder 
from agno.embedder.aws_bedrock import AwsBedrockEmbedder

agno_docs = UrlKnowledge(
    urls=["https://www.paulgraham.com/read.html"],
    # Use LanceDB as the vector database and store embeddings in the `agno_docs` table
    #vector_db=LanceDb(
    #    uri="tmp/pgdb",
    #    table_name="agno_docs",
    #    search_type=SearchType.hybrid,
    #    #embedder=OpenAIEmbedder(id="text-embedding-3-small")
    #    embedder=AwsBedrockEmbedder(id="cohere.embed-english-v3")
    #    embedder=AwsBedrockEmbedder(id="amazon.titan-embed-text-v2:0")
    #),
    vector_db=ChromaDb(
        persistent_client=True,
        path="tmp/chroma_db",
        collection="agno_docs",
        reranker=None,
        embedder=OpenAIEmbedder(id="text-embedding-3-small")
    )
)

knowledge_tools = KnowledgeTools(
    knowledge=agno_docs,
    think=True,
    search=True,
    analyze=True,
    add_few_shot=True,
)

In [107]:
knowledge_agent = Agent(
    agent_id="knowledge_agent",
    name="Knowledge Agent",
    model=model_ds,
    tools=[knowledge_tools],
    show_tool_calls=True,
    markdown=True,
    storage=SqliteStorage(
        table_name="knowledge_agent",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
)

reasoning_finance_team = Team(
    name="Reasoning Finance Team",
    mode="coordinate",
    model=model_qwen,
    members=[
        web_agent,
        finance_agent_2,
        thinking_tool_agent,
        cot_agent,
        reasoning_tool_agent,
        reasoning_model_agent,
    ],
    #reasoning=True,  # Does not work with model_qwen??
    tools=[ReasoningTools(add_instructions=True)],
    # uncomment it to use knowledge tools
    # tools=[knowledge_tools],
    team_id="reasoning_finance_team",
    debug_mode=True,
    instructions=[
        "Only output the final answer, no other text.",
        "Use tables to display data",
    ],
    markdown=True,
    show_members_responses=True,
    enable_agentic_context=True,
    add_datetime_to_instructions=True,
    success_criteria="The team has successfully completed the task.",
    storage=SqliteStorage(
        table_name="reasoning_finance_team",
        db_file=agent_storage_file,
        auto_upgrade_schema=True,
    ),
)

DEBUG Function: think registered with reasoning_tools

DEBUG Function: analyze registered with reasoning_tools

##### Issues:

1. Poor fault tolerance
2. Random error
3. Caching?

In [122]:
#Test team
reasoning_finance_team.print_response("Based on President Trump's recent social media posts and annoucement, recommend an investment portfolio by call ing out the companies and amount to invest for $100K with targetted return at 12.5% annual with risk tolerance at high level", stream=True)

┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Based on President Trump's recent social media posts and annoucement, recommend an investment portfolio by call ┃
┃ ing out the companies and amount to invest for $100K with targetted return at 12.5% annual with risk tolerance  ┃
┃ at high level                                                                                                   ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Web Search Agent Response ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Okay, I will search for recent social media posts and announcements by President Trump mentioning specific      ┃
┃ companies or industries. Given the current date is June 29, 2025, I will focus my search on posts from          ┃
┃ approximately the last 3-6 months (March - June 2025), prioritizing his activity on Truth Social, X (formerly   ┃
┃ Twitter), and any official statements released through his Save America PAC.                                    ┃
┃                                                                                                                 ┃
┃ Here's what I found as of 6:30 AM EST, June 29, 2025.  It's important to note that tracking every mention is    ┃
┃ difficult, but this represents a significant overview based on available reporting and direct platform          ┃
┃ searches.                                                                                                       ┃
┃                                                                                                                 ┃
┃ Companies/Industries Mentioned by President Trump (March - June 2025):                                          ┃
┃                                                                                                                 ┃
┃  • Boeing: (Multiple Mentions - April & May 2025) - Trump has repeatedly criticized Boeing’s leadership and     ┃
┃    quality control following several incidents with their aircraft. He specifically called for a change in      ┃
┃    management and suggested potential government contracts be reconsidered. (]8;id=865274;https://www.reuters.com/business/aerospace-defense/trump-renews-attacks-boeing-over-quality-issues-2025-05-15/\Source: Reuters - "Trump renews ]8;;\   ┃
┃    ]8;id=865274;https://www.reuters.com/business/aerospace-defense/trump-renews-attacks-boeing-over-quality-issues-2025-05-15/\attacks on Boeing over quality issues" - May 15, 2025]8;;\)                                                       ┃
┃  • Lockheed Martin: (April 2025) - Praised Lockheed Martin for its work on defense contracts, particularly      ┃
┃    related to missile defense systems, contrasting it favorably with Boeing. (]8;id=427818;https://thehill.com/policy/defense/4604278-trump-praises-lockheed-martin-renews-criticism-of-boeing/\Source: The Hill - "Trump praises]8;;\ ┃
┃    ]8;id=427818;https://thehill.com/policy/defense/4604278-trump-praises-lockheed-martin-renews-criticism-of-boeing/\Lockheed Martin, renews criticism of Boeing" - April 28, 2025]8;;\)                                               ┃
┃  • Apple: (June 2025) - Trump commented on Apple's potential manufacturing expansion within the United States,  ┃
┃    suggesting he would offer incentives if they brought more jobs back from overseas. (]8;id=910137;https://www.cnbc.com/2025/06/10/trump-says-hed-offer-incentives-for-apple-to-manufacture-more-in-us.html\Source: CNBC - "Trump ]8;;\  

DEBUG **** Team Run End: 4c4acc53-87dd-4fda-a264-49dea8f59976 ****

## Playground test

In [ ]:
playground = Playground(
    agents=[
        cot_agent,
        reasoning_tool_agent,
        reasoning_model_agent,
        knowledge_agent,
        thinking_tool_agent,
    ],
    teams=[reasoning_finance_team],
    name="Reasoning Demo",
    app_id="my-reasoning-demo",
    description="A playground for reasoning",
)

app = playground.get_app()

In [44]:
playground.serve(app="Reasoning Demo:app", host="0.0.0.0", port=8088, reload=True)

INFO Starting playground on http://0.0.0.0:8088

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Agent Playground ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                              ┃
┃                                                                              ┃
┃  Playground URL: ]8;id=821495;https://app.agno.com/playground?endpoint=0.0.0.0%3A8088/v1\https://app.agno.com/playground?endpoint=0.0.0.0%3A8088/v1]8;;\  ┃
┃                                                                              ┃
┃                                                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

DEBUG Creating app on Platform: Reasoning Demo, my-reasoning-demo

INFO:httpx:HTTP Request: POST https://api.agno.com/v2/apps "HTTP/1.1 202 Accepted"


DEBUG Agent app created: Reasoning Demo, my-reasoning-demo

INFO:     Will watch for changes in these directories: ['/opt/dlami/nvme/alfred/codes/agent/notebooks']
INFO:     Uvicorn running on http://0.0.0.0:8088 (Press CTRL+C to quit)
INFO:     Started reloader process [103803] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "Reasoning Demo".
INFO:watchfiles.main:3 changes detected
INFO:watchfiles.main:3 changes detected
INFO:     Stopping reloader process [103803]


## A research team for multiple agent reasoning with MCP

In [45]:
from agno.tools.mcp import MCPTools
from mcp import StdioServerParameters
from textwrap import dedent
from agno.agent import Agent
from agno.tools.mcp import MultiMCPTools

## Test **Optional**

In [138]:
prompt = "Tell me about multi-agentic reasoning based on the Github repos https://github.com/agno-agi/agno and https://github.com/langchain-ai/langgraph. You can analyze the entire repos for detail information."
messages=[
        {"role": "system", "content": "You are a helpful assistant. Please answer the user question accurately and truthfully. Also please make sure to think carefully before answering"},
        {"role": "user", "content": prompt},
    ],

In [139]:
# MultiMCPTools

async with MultiMCPTools(
    [
        "npx -y @modelcontextprotocol/server-github",
        "npx -y @modelcontextprotocol/server-sequential-thinking",
    ],
) as multi_mcp_tools:

    multi_agent = Agent(
        model=model_ds, # Model_openai work, other questionable??
        tools=[multi_mcp_tools],
        markdown=True,
        show_tool_calls=True,
        stream_intermediate_steps=True,
        #stream=True,
    )

    await agent.aprint_response(prompt, stream=True)  # Model_openai, qwen, ds uses prompt (str); others use messages (dict)

▰▰▱▱▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Tell me about multi-agentic reasoning based on the Github repos https://github.com/agno-agi/agno and            ┃
┃ https://github.com/langchain-ai/langgraph. You can analyze the entire repos for detail information.             ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (11.5s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ As an AI agent, I currently do not have the capability to access external websites or specific GitHub           ┃
┃ repositories directly, including the ones you've mentioned. However, I can provide a general overview based on  ┃
┃ common knowledge regarding multi-agentic reasoning and the typical content found in repositories related to     ┃
┃ such topics.                                                                                                    ┃
┃                                                                                                                 ┃
┃ ### Multi-Agentic Reasoning                                                                                     ┃
┃                                                                                                                 ┃
┃ Multi-agentic reasoning refers to the ability of multiple agents (which could be software agents, artificial    ┃
┃ intelligences, or even human agents) to interact, make decisions, and solve problems collaboratively. This      ┃
┃ approach is especially valuable in distributed systems, where agents may have different knowledge bases,        ┃
┃ perspectives, or objectives.                                                                                    ┃
┃                                                                                                                 ┃
┃ Key concepts in multi-agentic reasoning include:                                                                ┃
┃                                                                                                                 ┃
┃ 1. **Cooperation and Coordination**: Agents work together to achieve a common goal or share resources.          ┃
┃ Coordination protocols might be necessary to manage how agents work in parallel or sequentially.                ┃
┃                                                                                                                 ┃
┃ 2. **Communication**: Agents need a means to communicate effectively to share information and updates. This can ┃
┃ involve message-passing systems or shared knowledge bases.                                                      ┃
┃                                                                                                                 ┃
┃ 3. **Negotiation**: Agents may have competing objectives, leading to negotiation to reach mutually acceptable   ┃
┃ solutions.                                                                                                      ┃
┃                                                                                                                 ┃
┃ 4. **Intelligent Decision Making**: Agents employ reasoning techniques, such as game theory, for                ┃
┃ decision-making in uncertain or multidimensional situations.                                                    ┃
┃                                                                                                                 ┃
┃ 5. **Adaptability and Learning**: 

#### Hybrid MCP servers

In [147]:
server_params_github = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-github"],
    )
server_params_sthinking = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-sequential-thinking"],
    )

github_mcp_server_url = "http://agent.cavatar.info:8080/mcp"
arxiv_mcp_server_url = "http://infs.cavatar.info:8084/mcp"

# Create a client session to connect to the MCP server
#async with MultiMCPTools([server_params_github, server_params_sthinking]) as mcp_tools: #Doesn't work
async with (
    MCPTools(url=github_mcp_server_url, transport="streamable-http") as stream_github_mcp_tools,
    MCPTools(server_params=server_params_github ) as local_github_mcp_tools,
    MCPTools(server_params=server_params_sthinking ) as local_sthinking_mcp_tools,
    ):
    github_agent = Agent(
        model=model_qwen,  #model_qwen works, so does model_gemma but not model_nova nor model_openai_br
        tools=[local_github_mcp_tools, stream_github_mcp_tools, local_sthinking_mcp_tools],
        instructions=dedent("""\
            You are a GitHub assistant. Help users explore repositories and their activity.

            - Use headings to organize your responses
            - Output repo summary, url and star count
            - Be concise and focus on relevant information\
        """),
        markdown=True,
        show_tool_calls=True,
        debug_mode=False,
    )

    # Run the agent
    await github_agent.aprint_response(prompt, stream=True)

▰▱▱▱▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Search the latest github for multi-agentic reasoning  with the top 5 starred repos all dated in 2025.           ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Tool Calls ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ • search_repositories(query=multi-agentic reasoning created:2025, page=1, perPage=5)                            ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (19.9s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ ┃
┃ ┃                       Top 5 Starred Repositories for "Multi-Agentic Reasoning" (2025)                       ┃ ┃
┃ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃                                                    1. sirius                                                    ┃
┃                                                                                                                 ┃
┃ Owner: ]8;id=51996;https://github.com/zou-group\zou-group]8;;\                                                                                                ┃
┃ Stars: Not available in response                                                                                ┃
┃ URL: ]8;id=312455;https://github.com/zou-group/sirius\https://github.com/zou-group/sirius]8;;\                                                                        ┃
┃ Description: SiriuS: Self-improving Multi-agent Systems via Bootstrapped Reasoning                              ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                                                                                                 ┃
┃                                                  2. ReasonMed                                                   ┃
┃                                                                                                                 ┃
┃ Owner: ]8;id=516613;https://github.com/YuSun-Work\YuSun-Work]8;;\                                                                                               ┃
┃ Stars: Not available in response                                                                                ┃
┃ URL: ]8;id=798741;https://github.com/YuSun-Work/ReasonMed\https://github.com/YuSun-Work/ReasonMed]8;;\                                                                    ┃
┃ Description: ReasonMed: A 370K Multi-Agent Generated Dataset for Advancing Medical Reasoning                    ┃
┃                                             

### Expand to other agents with a Team

In [141]:
prompt = "Search the latest github for multi-agentic reasoning  with the top 5 starred repos all dated in 2025."

In [150]:
async with (
    MCPTools(url=github_mcp_server_url, transport="streamable-http") as stream_github_mcp_tools,
    MCPTools(url=arxiv_mcp_server_url, transport="streamable-http") as stream_arxiv_mcp_tools,
    MCPTools(server_params=server_params_github ) as local_github_mcp_tools,
    MCPTools(server_params=server_params_sthinking ) as local_sthinking_mcp_tools,
    ):
    ai_agent = Agent(
        model=model_qwen,  #model_qwen works, so does model_gemma but not model_nova nor model_openai_br
        tools=[local_github_mcp_tools, stream_github_mcp_tools, stream_arxiv_mcp_tools, local_sthinking_mcp_tools],
        instructions=dedent("""\
            You are a seasoned AI assistant. Help users explore arxiv ai and cs publications and github repositories and their activity.

            - Use headings to organize your responses
            - Be concise and focus on relevant information\
        """),
        markdown=True,
        show_tool_calls=True,
        debug_mode=False,
    )


    reasoning_ai_team = Team(
        name="Reasoning ai Team",
        mode="coordinate",
        model=model_qwen,  # Model_qwen without reasoning works
        members=[
            web_agent,
            ai_agent,
            #cot_agent,
            reasoning_tool_agent,
            reasoning_model_agent,
            thinking_tool_agent
        ],
        #reasoning=True, # Works with model_openai
        tools=[ReasoningTools(add_instructions=True)],
        # uncomment it to use knowledge tools
        # tools=[knowledge_tools],
        team_id="reasoning_ai_team",
        debug_mode=True,
        instructions=[
            "Only output the trusted answers from github or search.",
            "Aggregate and refine answers from multiple sources.",
            "Use tables to display data and chart display architecture flows.",
            "Output only the original and true source links without halluciantion.",
        ],
        markdown=True,
        show_members_responses=True,
        enable_agentic_context=True,
        add_datetime_to_instructions=True,
        success_criteria="The team has successfully completed the task.",
        storage=SqliteStorage(
            table_name="reasoning_ai_team",
            db_file=agent_storage_file,
            auto_upgrade_schema=True,
        ),
    )

    await reasoning_ai_team.aprint_response(prompt, stream=True, stream_intermediate_steps=True)

▰▰▰▱▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Search the latest github for multi-agentic reasoning  with the top 5 starred repos all dated in 2025.           ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Tool Calls ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ • transfer_task_to_member(member_id=agent-2, task_description=Search GitHub for repositories related to         ┃
┃ 'multi-agentic reasoning' with the highest stars, filtered for those created or updated in 2025. Retrieve the   ┃
┃ top 5 results with repository metadata including name, URL, stars, and description., expected_output=A list of  ┃
┃ 5 GitHub repositories with their names, URLs, star counts, and descriptions, sorted by stars and dated in       ┃
┃ 2025.)                                                                                                          ┃
┃ • transfer_task_to_member(member_id=agent2, task_description=Search GitHub for repositories related to          ┃
┃ 'multi-agentic reasoning' with the highest stars, filtered for those created or updated in 2025. Retrieve the   ┃
┃ top 5 results with repository metadata including name, URL, stars, and description., expected_output=A list of  ┃
┃ 5 GitHub repositories with their names, URLs, star counts, and descriptions, sorted by stars and dated in       ┃
┃ 2025.)                                                                                                          ┃
┃ • transfer_task_to_member(member_id=web-agent, task_description=Search the web for the latest GitHub            ┃
┃ repositories related to 'multi-agentic reasoning' with top 5 stars, dated in 2025. Retrieve repository names,   ┃
┃ URLs, star counts, and descriptions., expected_output=A list of 5 GitHub repositories with their names, URLs,   ┃
┃ star counts, and descriptions, sorted by stars and dated in 2025.)                                              ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (192.5s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Here are the top 5 GitHub repositories related to multi-agentic reasoning with the highest stars as of June     ┃
┃ 2025:                                                                                                           ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                     🏆 Top 5 Repositories (Sorted by Stars)                                     ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃   Rank   Repository Name               Stars   URL           Last Updated   Description                         ┃
┃  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  ┃
┃   1      ]8;id=475633;https://gith

DEBUG **** Team Run End: 726806f5-a990-4efd-a95c-2c3eb9164cbe ****

#### WIth team/agents reasoning on MCP servers with streamable-http protocol only

In [62]:
async with (
    MCPTools(url="http://agent.cavatar.info:8080/mcp", transport="streamable-http", timeout_seconds=80) as stream_mcp_github,
    MCPTools(url="http://infs.cavatar.info:8084/mcp", transport="streamable-http", timeout_seconds=80) as stream_mcp_arxiv
):
    github_agent = Agent(
        name="Github",
        role="Github Agent",
        model=model_openai, # Use model_gemma caused url link hallucination. Tool parser using 'Pythoinic' issues?? 
        tools=[stream_mcp_github],
        instructions=dedent("""\
            You are an agent that can find repos from github for fiven keywords.\
        """),
        debug_mode=True,
        add_datetime_to_instructions=True,
    )

    arxiv_agent = Agent(
        name="Arxiv",
        role="Arxiv Agent",
        model=model_openai,
        tools=[stream_mcp_arxiv],
        instructions=dedent("""\
            You are an agent that can find papers from arxiv for given keywords.\
        """),
        debug_mode=True,
        add_datetime_to_instructions=True,
    )

    reasoning_research_team2 = Team(
        name="Reasoning research for AI and machine learning",
        mode="coordinate",
        model=model_openai, # Other models including opensi_br_gw does not work for some reason why??!
        members=[
            web_agent,
            github_agent,
            arxiv_agent
        ],
        reasoning=True,
        tools=[ReasoningTools(add_instructions=True)],
        # uncomment it to use knowledge tools
        # tools=[knowledge_tools],
        team_id="reasoning_research_team",
        #role="Tech researcher",
        debug_mode=True,
        instructions=[
            "Only output the final answer, no other text.",
            "Use tables to display data",
        ],
        markdown=True,
        show_members_responses=True,
        enable_agentic_context=True,
        add_datetime_to_instructions=True,
        success_criteria="The team has successfully completed the task.",
        storage=SqliteStorage(
            table_name="reasoning_research_team",
            db_file=agent_storage_file,
            auto_upgrade_schema=True,
        ),
    )

INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: c58ae2bd2fd842d68cd746be352e404b
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-03-26
INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: GET http://agent.cavatar.info:8080/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: GET http://agent.cavatar.info:8080/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: POST http://agent.cavatar.info:8080/mcp/ "HTTP/1.1 200 OK"


DEBUG Function: get_repository_metadata registered with MCPTools

DEBUG Function: search_repositories registered with MCPTools

DEBUG Function: get_repository_topics registered with MCPTools

DEBUG MCPTools initialized with 3 tools

...

DEBUG Function: get_server_status registered with MCPTools

DEBUG MCPTools initialized with 4 tools

DEBUG Function: think registered with reasoning_tools

DEBUG Function: analyze registered with reasoning_tools

INFO:httpx:HTTP Request: DELETE http://infs.cavatar.info:8084/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: DELETE http://infs.cavatar.info:8084/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://agent.cavatar.info:8080/mcp "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: DELETE http://agent.cavatar.info:8080/mcp/ "HTTP/1.1 200 OK"


In [63]:
await reasoning_research_team2.aprint_response("Find the latest development in agentic reasoning from research pappers dated only in 2025. Output title, authors, date, abstracts and url in clickabke format",stream=True, stream_intermediate_steps=True)

▰▰▰▰▰▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Find the latest development in agentic reasoning from research pappers dated only in 2025. Output title,        ┃
┃ authors, date, abstracts and url in clickabke format                                                            ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Reasoning step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Identify Research Papers from 2025                                                                              ┃
┃ Action: I will search for research papers related to agentic reasoning published in 2025.                       ┃
┃ I have initiated a search for relevant papers published in 2025.                                                ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Reasoning step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Retrieve Latest Papers                                                                                          ┃
┃ Action: I will use an Arxiv search to find relevant papers about agentic reasoning published in 2025.           ┃
┃ I am preparing to make a search request to Arxiv for papers published in 2025.                                  ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Reasoning step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Define the Search Criteria                                                                                      ┃
┃ Action: I will specify the criteria for my search to focus on agentic reasoning in research papers published in ┃
┃ 2025.                                                                                                           ┃
┃ Criteria defined: focusing on agentic reasoning in research papers only from the year 2025.                     ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Reasoning step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Assign Search Task to Arxiv Agent                                                                               ┃
┃ Action: I will transfer the task of searching for research papers to the Arxiv Agent, specifying the criteria   ┃
┃ defined above.                                                                                                  ┃
┃ Task assigned to Arxiv Agent to search for relevant papers in 2025.                                             ┃
┃                                   

INFO:httpx:HTTP Request: POST https://api.agno.com/v1/telemetry/team-runs "HTTP/1.1 201 Created"


DEBUG **** Team Run End: 93572f2c-36a8-4a09-a2f4-d0143fb735bb ****

In [ ]:
# To avoid iterable issue as pecified in [this](https://stackoverflow.com/questions/72611531/adding-async-to-fastapi-function-gives-me-the-coroutine-object-is-not-iterabl)
async def runit(query:str, stream: bool=False, stream_intermediate_steps: bool=False):
    return await reasoning_research_team.aprint_response(query, stream=stream, stream_intermediate_steps=stream_intermediate_steps)

await runit("Find the latest development in agentic reasoning from research pappers and code samples all dated in 2025.", True, True)

### Using playgoround

In [39]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    asyncio.run(agno_docs.aload(recreate=True))
    playground.serve(app="reasoning-demo:app", host="0.0.0.0", port=8088, reload=True)
    #playground.serve(app="agno_mcp_reasoning:app", reload=True)

# reasoning_tool_agent
# Solve this logic puzzle: A man has to take a fox, a chicken, and a sack of grain across a river.
# The boat is only big enough for the man and one item. If left unattended together,
# the fox will eat the chicken, and the chicken will eat the grain. How can the man get everything across safely?


# knowledge_agent prompt
# What does Paul Graham explain here with respect to need to read?

INFO Dropping collection

DEBUG Creating Persistent Chroma Client

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


DEBUG Deleting collection: agno_docs

DEBUG Collection does not exist: Collection [agno_docs] does not exists

...

INFO:httpx:HTTP Request: GET https://www.paulgraham.com/read.html "HTTP/1.1 200 OK"


DEBUG Filtering out existing documents before insertion.

DEBUG Inserting 1 documents

INFO:httpx:HTTP Request: POST http://bedroc-proxy-mjagql6uvtpz-789127345.us-west-2.elb.amazonaws.com/api/v1/embeddings "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'detail': 'An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #/texts/0: expected maxLength: 2048, actual: 4993, please reformat your input and try again.'}

In [4]:
async def main():
    # 1. MCP server command for Context7 (public library docs)
    command = "npx -y @upstash/context7-mcp@latest"
    
    # 2. Launch the MCP server as a subprocess and initialize tools
    async with MCPTools(command) as mcp_tools:
        # 3. Create an Agno agent with an LLM and the MCP tools
        agent = Agent(
            name="Agno Context7 Doc Agent",
            role="An AI agent that provides up-to-date library documentation "
                 "and code snippets using Context7 MCP.",
            model=OpenAIChat(id="gpt-4"),  # using GPT-4 via OpenAI API
            tools=[mcp_tools],
            instructions='''
You are a programming assistant with access to Context7 (a tool that fetches live documentation).
When a user asks for documentation:
1. Identify the library name (the first word/phrase of the query).
2. Use the `resolve-library-id` tool with that name to get the internal library ID.
3. Take the rest of the query as the documentation topic.
4. Use the `get-library-docs` tool with the library ID and topic to fetch relevant docs.
5. Limit results to about 5000 tokens (unless the user asks for a different amount).
6. Present the information clearly with any code snippets.
'''
        )
        # 4. Use the agent to answer a question (as an example)
        question = "Agno MCP tools usage"  # e.g., user asks about "Agno MCP tools"
        answer = await agent.run(question)
        print(answer)

Concretely, to use the Context7 MCP server as the backend for library Q&A, we would modify the Streamlit app’s code as follows (in pseudocode terms):

Import and configure MCPTools:

In [10]:
from agno.tools.mcp import MCPTools 
context7_cfg = {"command": "npx", 
                "args": ["-y", "@upstash/context7-mcp@latest"]
                }

Sets up the command that will launch the Context7 server. You can define this config at the top of your app script. (Using a dict of command and args is equivalent to the single string we used earlier, Agno can accept either. The -y ensures npm installs without prompts.

Initialize MCPTools and incorporate into agents:
If our app created agents on the fly for each query, we could do:

In [12]:
async with MCPTools(**context7_cfg) as mcp_tools:
    doc_agent = Agent(
        name="DocAgent",
        model=model,
        tools=[mcp_tools],
        # perhaps other agents like web_agent, etc.
    )
    team = Agent(
        team=[doc_agent, ...],
        instructions="Cooperate to answer the user."
    )
    result = await team.run(user_query)

TypeError: Toolkit.__init__() got an unexpected keyword argument 'args'

In [43]:
import asyncio
import time

async def a(time: int):
    print(f'function a: sleeping for {time} seconds')
    await asyncio.sleep(time)
    print(f'function a: sleeping for {time} completed')
    return time

start_time = time.time()
arr = await asyncio.gather(a(2), a(6), a(4))
end_time = time.time()
print(f'Time taken: {end_time - start_time} seconds')
print(f'arr: {arr}')

function a: sleeping for 2 seconds
function a: sleeping for 6 seconds
function a: sleeping for 4 seconds
function a: sleeping for 2 completed
function a: sleeping for 4 completed
function a: sleeping for 6 completed
Time taken: 6.0032713413238525 seconds
arr: [2, 6, 4]
